In [1]:
# Import libraries
import random
import math 
import numpy as np

# Import timings
import time

# Importing Pandas to create DataFrame
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt

import seaborn as sns

from collections import Counter
from collections import defaultdict
from scipy.stats import gaussian_kde

#from neo4j import GraphDatabase


# Define a class for the neo4j session
class Neo4jSession:
    def __init__(self):
        self._driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "simon32_graph"))
        
    def close(self):
        self._driver.close()
    
    # Create a Neo4j session
    neo4j_session = Neo4jSession()

In [2]:
def create_dataframe():
    # Create empty DataFrame with specific column names & types
    df = pd.DataFrame({'Best weight': pd.Series(dtype='int'),
                       'Duration': pd.Series(dtype='float'),
                       'Iterations': pd.Series(dtype='int')})
    df.index.rename('Experiment', inplace=True)

    df["Best weight"] = df["Best weight"].astype(int)

    df["Iterations"] = df["Iterations"].astype(int)

    return df

In [3]:
# Create a highwaylist function that adds the differential characteristics to corresponding lists
def highwaylist():
#    global stratum
    # Create 4 lists to hold the differential characteristics of each file
    alph,bet,gam,pro = [],[],[],[]
    with open('alph.txt') as ip1, open('bet.txt') as ip2, open('gam.txt') as ip3, open('pro.txt') as ip4:
        for i1,i2,i3,i4 in zip(ip1,ip2,ip3,ip4):
            # Append each differential to theappropriate list and remove whitespaces
            hw.append(NMTS(int(i1.strip()),int(i2.strip()),int(i3.strip()),float(i4.strip()))) 
            


In [4]:
# Function to plot the distribution of output differentials
def diff_distribution():
    differential_distribution = {}

    alph_data = [item.dx for item in hw]
    bet_data = [item.dy for item in hw]
    gam_data = [item.dz for item in hw]
    pro_data = [item.wt for item in hw]

    for i, (alph, bet, gam, pro) in enumerate(zip(alph_data, bet_data, gam_data, pro_data)):
        input_difference = alph ^ bet
        if input_difference not in differential_distribution:
            differential_distribution[input_difference] = []
        differential_distribution[input_difference].append((gam, pro))
        
    mean_weights = {key: np.mean(np.array(val)[:, 1]) for key, val in differential_distribution.items()}

    input_differences = list(mean_weights.keys())
    mean_weights = list(mean_weights.values())


    #filtered_input_differences = [key for key in input_differences if -50 <= key <= 50]
    #filtered_mean_weights = [mean_weights[key] for key in filtered_input_differences]
    bar_colors = ['blue']
    plt.gcf().set_facecolor("#4d1111")
    plt.figure(figsize=(10, 6))
    plt.bar(input_differences, mean_weights, color=bar_colors)
    plt.xlabel('Input difference')
    plt.ylabel('mean weight')
    plt.title('Differential Distribution')
    plt.grid(True)
    #plt.tight_layout()
    plt.show()

In [5]:
def sample_data(hw):
    # Set the sample size as a decimal percentage
    samepl_size = 0.05
    
    # group differentials by the dz attribute values
    grouped_data = defaultdict(list)
    
    for item in hw:
        grouped_data[item.dz].append(item) # store the whole item
    
    # create an empty list
    final_nmts_list = []
    
    # Sample from each differential based on dz samples
    for dz, group in grouped_data.items():
        group_size = len(group)
        num_samples = max(1, math.ceil(group_size * samepl_size)) # Ensures at least 1 of each differential if it is too small
        
        for item in group[:num_samples]:
            nmts_instance = NMTS(dx=item.dx, dy=item.dy, dz=item.dz, wt=item.wt)
            final_nmts_list.append(nmts_instance)
    
   
    print(len(final_nmts_list))
    
    
    
    
    return final_nmts_list

In [6]:
def plot_distribution():
    global stratum
    # plot original distribution
    dz_values = [item.dz for item in hw]
    plt.hist(dz_values, bins=50, alpha=0.9, label="Original Distribution", color="#bdc9e1") 
    
    # plot sample distribution
    sampled_dz_values = [stratum]
    #plt.hist(sampled_dz_values, bins=50, alpha=0.9, label="Samples Distribution", color="#045a8d")  
    
    #Plot histogram and store output
    counts, bins, patches = plt.hist(sampled_dz_values, bins=50, alpha=0.9, label="Samples Distribution", color="#045a8d")
    
    # Get maximum bin count
    max_bin_count = max(counts)
    
    # Plot data
    plt.title("Distribution of output differentials: Original vs Stratified sample")
    plt.xlabel("Output differentials")
    plt.ylabel("Frequency")
    plt.legend(loc="upper right")
    plt.yticks([max_bin_count, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000])
    plt.axhline(y=max_bin_count, color="r", linestyle="--", label=("Y=1000 Line"))
    
    # Generate plot
    plt.show() 
    

In [7]:
def mask_1(a,b,wshift):
    #initialise variables s1 and var. s1 variable is a flag to control the loop and var calculates the new mask
    s1=0
    var=0
    
    # Continue the loop as long as s1 equals 0
    while(s1==0):
        
        #shift the cariables a and b to the left by wshift bits value
        # Then perform a bitwise AND operation with the global mask. The bitwise AND is defined by the &
        a1=(a<<(wshift))&mask
        b1=(b<<(wshift))&mask
        
        # Perform a bitwise OR operation on a1 and b1 variables and store the result in s1
        s1=a1+b1
        
        # If s1 is still equal to zero (indicating no set bits in a1 or b1),
        # increment var variable and decrement wshift by 1 for the next iteration of the loop
        if(s1==0):
            var=var+1
        wshift=wshift-1
        
        # Calculate mask1 by shifting global mask to the left by var bits
        # then performing bitwise AND with the global mask
        # Ths creates a new mask that zeroes out the var least significant bits
        mask1=(mask<<var)&mask
        
    # Return the newly calculated mask
    return mask1

In [8]:

def weightAND(alpha1,beta1,gamma1): 
    # Perform a bitwise AND operation between gamma1 and the bitwise negative of the XOR between aplha1 and beta 1 and assign to the s variable
    # One done, apply the global mask
    s=(gamma1&(~(alpha1^beta1)))&mask
    
    # Convert the bitwise AND between the XOR of alhpa1 and beta1 and the global mask to a minary string (bin function)
    temp=bin((alpha1^beta1)&mask)
    
    # Calculate the hamming weight as the count of "1's" in the binary string ignoring the first character which is "0b" for
    wt=(temp[1:].count("1"))
    
    # If s is not zero return -200 and 200
    # This could be a case where the differential characteristic is not possible or not considered
    if(s!=0):        
        return -200,200
    else:
        # If s equals zero return the hamming weight and the probability of the differential characteristic
        # The probability is calculated as 2 to the power of negative weight
        return wt,math.pow(2,-wt)

In [9]:
#weightAND(0x4004,0x220,0x0)

In [10]:
#right circular shifts
def ROR(x,r):
    # Left shift x by the value of SIMON_TYPE - r places, moving the least significant bits to the most significant bit positions
    # Next, right shift x by r places, moving the r most significant bits to te least significant bit positions
    # Add the 2 value togather to rotate the bits of x a total of r places to the right
    # Apply the mask to ensure the result is within the desired bit size
    x = ((x << (SIMON_TYPE - r)) + (x >> r)) & mask
    return x

#left circular shifts
def ROL(x,r):
    # Right shift x by the value of SIMON_TYPE - r places, moving the least significant bits to the most significant bit positions
    # Next, left shift x by r places, moving the r most significant bits to te least significant bit positions
    # Add the 2 value togather to rotate the bits of x a total of r places to the left
    # Apply the mask to ensure the result is within the desired bit size
    x = ((x >> (SIMON_TYPE - r)) + (x << r)) & mask
    return x

In [11]:
# Define a find differential path function
# The fuction holds 4 arguments
# st1 and st0 are the initial states of the input and output
# the SIMON_ROUNDS is the number of rounds
# n counter variable
def find_diff_path(st1,st0,SIMON_ROUNDS,n):
    #stratum = sample_data()
    #global stratum
    # Initialise local variables  
    
    # Initialise the temporary hamming weight of the differential path
    temp_wt=0 
    
    # Initialise a temporary list to store the differential characteristics
    tempdec_list=[]
    
    # Loop for the number of defined SIMON_ROUNDS
    for i in range (0,SIMON_ROUNDS):
        
        # Calculate A, B and C values by rotating st1
        A=ROL(st1,alpha) # left circular shift hw[ch].dx (alph.txt) by 1
        B=ROL(st1,beta)  # left circular shift hw[ch].dx (alph.txt) by 8
        C=ROL(st1,gamma) # left circular shift hw[ch].dx (alph.txt) by 2
        
        # Initialise hamming weight as -200
        wt=-200
        
        # Loop through the wt variable until a non-negative weight is found
        while(wt==-200):
            # Take a random characteristic from the hw variable
            #op=hw[random.randint(0,len(hw)-1)].dz
            
            #op = random.choice(stratum).dz
            op = random.choice(stratum)
            
            # Calculate the hamming weights using left shift A, left shift B and OP
            wt,wt1=weightAND(A,B,op)
            
        # Calculate the value of D from the XOR of op and C
        D=op^C
        
        # Calculate the value of E from the XOR of st0 and D
        E=st0^D
        
        # Add the new charactristic to the list and update the total haming weight
        tempdec_list.append(NMTS(st1,st0,op,wt)) 
        temp_wt= temp_wt+tempdec_list[i].wt
        
        #update state with new valid output differential
        st0=st1; st1=E
        
    # Increment the counter by 1
    n=n+1
    
    # Return the list of characteristics, total hamming weight and counter
    return tempdec_list, temp_wt, n



In [12]:
# Define the find_best_path function
# Takes 5 arguments
# st1 and st0 are the initial states of the input and output of the block cypher
# SIMON_ROUNDS is the number of rounds
# wt_above is the cumulative hamming weight from previous iterations
# best_weight is the lowest hamming weight so far
def find_best_path(st1,st0, SIMON_ROUNDS, wt_above, best_wt):

    #using n as index value for list and initialise to 0
    n=0
    
    # Loop through SIMON_ROUNDS beginning at the end and then decrement by one until it reaches the stop value of 0
    for r in range(SIMON_ROUNDS,0,-1):
        
        # Find differential path for the round
        tempdec_list, temp_wt, n = find_diff_path(st1,st0,r, n)
        
        # Check if the hamming weight for the path found is less than the best hamming weight so far
        if((temp_wt+wt_above) < best_wt):
            
            # If the hamming weight is less than the best hamming weight so far update the best hamming weight
            best_wt= temp_wt+wt_above
            
            # Update the dec_list with the new differential path
            for i,j in zip(range(n-1,22), range(len(tempdec_list))):
                dec_list[i]=(NMTS(tempdec_list[j].dx,tempdec_list[j].dy,tempdec_list[j].dz,tempdec_list[j].wt))
        
        # If n is less than SIMON_ROUNDS update st1 and st0 for the next iteration
        if(n<SIMON_ROUNDS):
            # Update to the dx and dy values of the nth dec_list item
            st1=dec_list[n].dx
            st0=dec_list[n].dy
            
        # Add the weight of the last character to the cumulative wt_above variable
        wt_above= wt_above+dec_list[n-1].wt
        
    # Return the best hamming weight found
    return best_wt

In [13]:
class NMTS(object):
    """__init__() functions as the class constructor"""
    def __init__(self, dx=None, dy=None, dz=None, wt=None):
        self.dx = dx # Assign the input dx value to the objects dx attribute
        self.dy = dy # Assign the input dy value to the objects dy attribute
        self.dz = dz # Assign the input dz value to the objects dz attribute
        self.wt = wt # Assign the input wt value to the objects wt attribute 

In [14]:
# Define Neo4j transaction functions
def create_node(tx, dx, dy, dz, wt):
    tx.run("CREATE (n:None {dx: $dx, dy: $dy, dz: $dz, wt: $wt})", dx=dx, dy=dy, dz=dz, wt=wt)

def create_relationships(tx, source_dz, target_dz):
    tx.run("MATCH (source:Node {dz: $source_dz}), (target:Node {dz: $target_dz})"
          "CREATE (source)-[:NEXT]->(target)", source_dz=source_dz, target_dz=target_dz)

In [ ]:
# Define a dictionary to hold row names
row_names = {}



target_weight = 32

#############################
# Initialise hw list
hw=[] 


#nested loop to run number of time
highwaylist()


diff_distribution()

# set experiment counter to 0
j = 0

# Set number of experiments
experiment_loops = 193

# Create dataframe and define to df variable
df = create_dataframe()

# call the sample data function to get a list of the NMTS objects
nmts_objects = sample_data(hw)

# Extract sample dz values from the NMTS objects to use as the quota stratum
stratum = [obj.dz for obj in nmts_objects]

# Loop the code x times to simulate x number of experiments
for j in range(experiment_loops):

    # set iterations counter to 0
    iterations = 0
    
    

    # Left shift 1, left shift 8, left shift 2 - Defined as the SIMON round function
    alpha, beta, gamma = 1,8,2

    # Define SIMON ROUNDS to attack
    SIMON_ROUNDS=10 


    # Define SIMON type
    SIMON_TYPE=16

    # Define mask
    mask = 2 ** SIMON_TYPE - 1

    # Initialise wshift to 15 which is the maximum possible shift in a 16 bit number
    wshift=15

    # define decryption list of 22 items of 0
    dec_list=[0]*22

    # Initialise wt_above to 0
    wt_above=0

    # Initialise best_wt
    best_wt=999

    # initialise s to high value, eg. 9999 to accept best_wt
    s=9999


    # initialise ch as current round position as it iterates each round
    ch = 2

    # Start the timer for the experiment
    start_time = time.time()
    while(best_wt >= target_weight and ch < 30200): # Added iteration cutoff at 75% original (IQR3)

        # Find the best differential path in forward direction for the current hw and assign to best_wt
        best_wt=find_best_path(hw[ch].dx,hw[ch].dy,SIMON_ROUNDS,wt_above,best_wt)
        ch=ch+1
        iterations = iterations + 1
        # If the total weight is less than the smallest weight found so far update the smallest and print the path
        if(best_wt<s):
            s=best_wt
            print(s,ch)

            # If best_wt is less than 33 print the decription list
            if(best_wt<33):
                print("Dec list is:")   
                for i in range(0,SIMON_ROUNDS):    
                    # Print input, output, differential and weigth
                    print(hex(dec_list[i].dx), hex(dec_list[i].dy),hex(dec_list[i].dz),(dec_list[i].wt))
                    
                    
                    

    # Print the best weight                
    print("Best weight is:",best_wt) 

    # Define end time
    end_time = time.time()

    # define elapsed time
    elapsed_time = end_time - start_time

    # Print the time taken
    print("Elapsed time: ", elapsed_time)

    df.loc[len(df.index)] = [int(best_wt), elapsed_time, ch]

    # Set the current row name
    row_names[j] = "Experiment " + str(j + 1)

    # check if it is the last iteration and plot the distribution
    if j == experiment_loops - 1:
        plot_distribution()


# rename row names
df = df.rename(index = row_names)   


# add the mean, median, min and max to the table
df.loc["MEAN"] = [df["Best weight"].mean(), df["Duration"].mean(), df["Iterations"].mean()]
df.loc["MEDIAN"] = [df["Best weight"].median(), df["Duration"].median(), df["Iterations"].median()]
df.loc["MIN"] = [df["Best weight"].min(), df["Duration"].min(), df["Iterations"].min()]
df.loc["MAX"] = [df["Best weight"].max(), df["Duration"].max(), df["Iterations"].max()]

# save the csv file 
file_name = f"./simon32_oneway.csv"
df.to_csv(file_name, encoding='utf-8', header='true')

#df
display(df)
df.drop(df.index, inplace=True)





import networkx as nx
G = nx.DiGraph()

for n in range(len(stratum) - 1):
    G.add_edge(stratum[n], stratum[n+1])
pos = nx.spring_layout(G)
nx.draw_networkx(G, pos, with_labels=True, node_size=500, node_color="blue", node_shape="o", alpha=0.5, linewidths=40)
plt.axis('off')
plt.show()